In [1]:
from datetime import datetime, timedelta
from statsmodels.stats import weightstats
import itertools as itr
import numpy as np
import pandas as pd
from numpy.random import randint
import re
import collections as col
from IPython.core.display import display, HTML
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot, download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)
import prepare_data

In [2]:
df = pd.read_csv('main_task.csv')    # загрузим
df.columns = ['id', 'city', 'cuisine', 'ranking', 'rating',
              'price_range', 'rev_nums', 'reviews', 'url', 'id_ta']  # переименуем
# eсли имеются дупликаты в датасете по id трипадвизора -дропнем эти дупликаты
df = df.drop_duplicates(subset=['id_ta']).reset_index().drop(['index'], axis=1)
# также дропну слобцы с которыми не буду работать при подготовке модели
df = df.drop(columns=['id', 'id_ta', 'reviews'], axis=1)
display(df.info())                   # действительно было  20 дупликатов с сете
df=pd.concat([df.drop(['rating'],axis=1),df['rating']],axis=1)
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39980 entries, 0 to 39979
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         39980 non-null  object 
 1   cuisine      30701 non-null  object 
 2   ranking      39980 non-null  float64
 3   rating       39980 non-null  float64
 4   price_range  26101 non-null  object 
 5   rev_nums     37437 non-null  float64
 6   url          39980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 2.1+ MB


None

,city,cuisine,ranking,price_range,rev_nums,url,rating
0,Paris,"['European', 'French', 'International']",5570.0,$$ - $$$,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,3.5
1,Stockholm,NaN,1537.0,NaN,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,4.0
2,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,$$$$,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...,4.5
3,Berlin,NaN,3458.0,NaN,3.0,/Restaurant_Review-g187323-d1358776-Reviews-Es...,5.0
4,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,$$ - $$$,84.0,/Restaurant_Review-g187309-d6864963-Reviews-Au...,4.0
...,...,...,...,...,...,...,...
39975,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,$$ - $$$,79.0,/Restaurant_Review-g187849-d2104414-Reviews-Ro...,4.5
39976,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,$$ - $$$,542.0,/Restaurant_Review-g187147-d1800036-Reviews-La...,3.5
39977,Stockholm,"['Japanese', 'Sushi']",1652.0,NaN,4.0,/Restaurant_Review-g189852-d947615-Reviews-Sus...,4.5
39978,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,$$ - $$$,70.0,/Restaurant_Review-g274856-d1100838-Reviews-Ho...,4.0


# Начнем заполнять пропуски и преобразовывать столбцы

In [3]:
# преобразуем кухни в списки строковых величин
df['cuisine'] = df['cuisine'].apply(
    lambda x: x[1:-1].replace("'", "") if pd.notna(x) else None)
df['cuisine'] = df['cuisine'].apply(lambda x: x.replace(
    " ", "") if pd.notna(x) else None)   # также уберем пробелы после запятых
# в столбце price_range 3 значение диапазонов цен
prices = list(df['price_range'].value_counts().index)
df['price_range'] = df['price_range'].apply(lambda x: 0 if x == prices[1] else 1 if x ==
                                            prices[2] else 2 if x == prices[0] else np.NaN)  # преобразуем их в high,low,mid

In [3]:
ind=list(df['price_range'].value_counts().index)
lenght = len(df['price_range'].value_counts().index)
val = [i for i in range(lenght)]
display(ind)
val[ind.index('$$$$')]


['$$ - $$$', '$', '$$$$']

2

In [7]:
df['price_range'].value_counts()

$$ - $$$    28243
$            9541
$$$$         2196
Name: price_range, dtype: int64

In [5]:
prepare=prepare_data.PrepareData()
num_cols=prepare.num_cols(df,len(df['city'].value_counts()))
bin_cols=prepare.bin_cols(df)
cat_cols=prepare.cat_cols(df,len(df['city'].value_counts()))

In [6]:
df=prepare.fill_na(df)
df

,city,cuisine,ranking,price_range,rev_nums,url,rating
0,Paris,"['European', 'French', 'International']",5570.0,$$ - $$$,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,3.5
1,Stockholm,['French'],1537.0,$$ - $$$,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,4.0
2,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,$$$$,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...,4.5
3,Berlin,['Italian'],3458.0,$$ - $$$,3.0,/Restaurant_Review-g187323-d1358776-Reviews-Es...,5.0
4,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,$$ - $$$,84.0,/Restaurant_Review-g187309-d6864963-Reviews-Au...,4.0
...,...,...,...,...,...,...,...
39975,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,$$ - $$$,79.0,/Restaurant_Review-g187849-d2104414-Reviews-Ro...,4.5
39976,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,$$ - $$$,542.0,/Restaurant_Review-g187147-d1800036-Reviews-La...,3.5
39977,Stockholm,"['Japanese', 'Sushi']",1652.0,$$ - $$$,4.0,/Restaurant_Review-g189852-d947615-Reviews-Sus...,4.5
39978,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,$$ - $$$,70.0,/Restaurant_Review-g274856-d1100838-Reviews-Ho...,4.0


In [11]:
df=prepare.tread_categorial(df,bin_cols+cat_cols)
df

,city,cuisine,ranking,price_range,rev_nums,url,rating
0,1,"['European', 'French', 'International']",5570.0,0,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,3.5
1,15,['French'],1537.0,0,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,4.0
2,0,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,2,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...,4.5
3,4,['Italian'],3458.0,0,3.0,/Restaurant_Review-g187323-d1358776-Reviews-Es...,5.0
4,13,"['German', 'Central European', 'Vegetarian Fri...",621.0,0,84.0,/Restaurant_Review-g187309-d6864963-Reviews-Au...,4.0
...,...,...,...,...,...,...,...
39975,5,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,0,79.0,/Restaurant_Review-g187849-d2104414-Reviews-Ro...,4.5
39976,1,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,0,542.0,/Restaurant_Review-g187147-d1800036-Reviews-La...,3.5
39977,15,"['Japanese', 'Sushi']",1652.0,0,4.0,/Restaurant_Review-g189852-d947615-Reviews-Sus...,4.5
39978,17,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,0,70.0,/Restaurant_Review-g274856-d1100838-Reviews-Ho...,4.0


In [4]:
grouped=df.groupby(['city','price_range'])['price_range'].count()   # сгруппируем и посчитаем сколько в каких городах ресторанов каких ценовых категорий
grouped

city       price_range
Amsterdam  0.0            163
           1.0             31
           2.0            644
Athens     0.0            137
           1.0             18
                         ... 
Warsaw     1.0             28
           2.0            299
Zurich     0.0             47
           1.0             34
           2.0            293
Name: price_range, Length: 93, dtype: int64

In [5]:
# сгруппируем и посчитаем сколько в каких городах ресторанов каких ценовых категорий
grouped = df.groupby(['city', 'price_range'])['price_range'].count()
grouped

city       price_range
Amsterdam  0.0            163
           1.0             31
           2.0            644
Athens     0.0            137
           1.0             18
                         ... 
Warsaw     1.0             28
           2.0            299
Zurich     0.0             47
           1.0             34
           2.0            293
Name: price_range, Length: 93, dtype: int64

In [6]:
"""заполним вторым уровнем мул наиболее часто встречающимся в городе,содержащим ценовой диапазон если пропуск ,иначе оставим значение"""
df['price_range'] = pd.Series(df.index).apply(lambda x: grouped.loc[df['city'].loc[x], :].idxmax()[
    1] if pd.isna(df['price_range'].loc[x]) else df['price_range'].loc[x])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39980 entries, 0 to 39979
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         39980 non-null  object 
 1   cuisine      30701 non-null  object 
 2   ranking      39980 non-null  float64
 3   rating       39980 non-null  float64
 4   price_range  39980 non-null  float64
 5   rev_nums     37437 non-null  float64
 6   url          39980 non-null  object 
dtypes: float64(4), object(3)
memory usage: 2.1+ MB


# посчитаем среднее количество отзывов в различных городах и определенных ценовых группах

In [7]:
grouped = df.groupby(['city', 'price_range'])['rev_nums'].mean()
grouped

city       price_range
Amsterdam  0.0            125.070513
           1.0            455.935484
           2.0            139.297998
Athens     0.0             57.201493
           1.0            243.375000
                             ...    
Warsaw     1.0            220.857143
           2.0             58.185606
Zurich     0.0             41.851064
           1.0            208.529412
           2.0             71.014151
Name: rev_nums, Length: 93, dtype: float64

In [8]:
"""заполним средним значением по городу и ценовой категории если пропуск , иначе оставим значение"""
df['rev_nums'] = pd.Series(df.index).apply(lambda x: grouped.loc[df['city'].loc[x],
                                                                 df['price_range'].loc[x]] if pd.isna(df['rev_nums'].loc[x]) else df['rev_nums'].loc[x])

In [9]:
df['cuisine'] = df['cuisine'].apply(lambda x: x.split(
    ',') if pd.notna(x) else None)  # кухни пердставим в виде списков
cuisine = df['cuisine'].dropna(axis=0)
# а пропуски заполним рандомными занчениями из заполненных строк
df['cuisine'] = df['cuisine'].apply(
    lambda x: cuisine.iloc[randint(0, len(cuisine))] if x == None else x)
df

,city,cuisine,ranking,rating,price_range,rev_nums,url
0,Paris,"[European, French, International]",5570.0,3.5,2.0,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...
1,Stockholm,"[Bar, British, Pub, Grill, VegetarianFriendly]",1537.0,4.0,2.0,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...
2,London,"[Japanese, Sushi, Asian, Grill, VegetarianFrie...",353.0,4.5,1.0,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...
3,Berlin,[Cafe],3458.0,5.0,2.0,3.0,/Restaurant_Review-g187323-d1358776-Reviews-Es...
4,Munich,"[German, CentralEuropean, VegetarianFriendly]",621.0,4.0,2.0,84.0,/Restaurant_Review-g187309-d6864963-Reviews-Au...
...,...,...,...,...,...,...,...
39975,Milan,"[Italian, VegetarianFriendly, VeganOptions, Gl...",500.0,4.5,2.0,79.0,/Restaurant_Review-g187849-d2104414-Reviews-Ro...
39976,Paris,"[French, American, Bar, European, VegetarianFr...",6341.0,3.5,2.0,542.0,/Restaurant_Review-g187147-d1800036-Reviews-La...
39977,Stockholm,"[Japanese, Sushi]",1652.0,4.5,2.0,4.0,/Restaurant_Review-g189852-d947615-Reviews-Sus...
39978,Warsaw,"[Polish, European, EasternEuropean, CentralEur...",641.0,4.0,2.0,70.0,/Restaurant_Review-g274856-d1100838-Reviews-Ho...


In [10]:
cuisines = []
for el in cuisine:
    cuisines.extend(el)
pd.Series(cuisines).value_counts().head(15)

VegetarianFriendly    11183
European              10056
Mediterranean          6271
Italian                5963
VeganOptions           4482
GlutenFreeOptions      4110
Bar                    3296
French                 3189
Asian                  3010
Pizza                  2849
Spanish                2788
Pub                    2449
Cafe                   2325
FastFood               1705
British                1595
dtype: int64

In [11]:
for el in pd.Series(cuisines).value_counts().index:
    df[el]=df['cuisine'].apply(lambda x : 1 if el in x else 0)
df
    

,city,cuisine,ranking,rating,price_range,rev_nums,url,VegetarianFriendly,European,Mediterranean,...,Caucasian,Azerbaijani,Polynesian,Welsh,Fujian,Xinjiang,Salvadoran,Yunnan,Latvian,Burmese
0,Paris,"[European, French, International]",5570.0,3.5,2.0,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Stockholm,"[Bar, British, Pub, Grill, VegetarianFriendly]",1537.0,4.0,2.0,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,London,"[Japanese, Sushi, Asian, Grill, VegetarianFrie...",353.0,4.5,1.0,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Berlin,[Cafe],3458.0,5.0,2.0,3.0,/Restaurant_Review-g187323-d1358776-Reviews-Es...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Munich,"[German, CentralEuropean, VegetarianFriendly]",621.0,4.0,2.0,84.0,/Restaurant_Review-g187309-d6864963-Reviews-Au...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39975,Milan,"[Italian, VegetarianFriendly, VeganOptions, Gl...",500.0,4.5,2.0,79.0,/Restaurant_Review-g187849-d2104414-Reviews-Ro...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
39976,Paris,"[French, American, Bar, European, VegetarianFr...",6341.0,3.5,2.0,542.0,/Restaurant_Review-g187147-d1800036-Reviews-La...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
39977,Stockholm,"[Japanese, Sushi]",1652.0,4.5,2.0,4.0,/Restaurant_Review-g189852-d947615-Reviews-Sus...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39978,Warsaw,"[Polish, European, EasternEuropean, CentralEur...",641.0,4.0,2.0,70.0,/Restaurant_Review-g274856-d1100838-Reviews-Ho...,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# создадим dummie-переменные до ценовому диапазону
dummies = pd.get_dummies(df['price_range'])
df = pd.concat([df, dummies], axis=1)
dummies = pd.get_dummies(df['city'])
df = pd.concat([df, dummies], axis=1)
df

,city,cuisine,ranking,rating,price_range,rev_nums,url,VegetarianFriendly,European,Mediterranean,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,Paris,"[European, French, International]",5570.0,3.5,2.0,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,Stockholm,"[Bar, British, Pub, Grill, VegetarianFriendly]",1537.0,4.0,2.0,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,London,"[Japanese, Sushi, Asian, Grill, VegetarianFrie...",353.0,4.5,1.0,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Berlin,[Cafe],3458.0,5.0,2.0,3.0,/Restaurant_Review-g187323-d1358776-Reviews-Es...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Munich,"[German, CentralEuropean, VegetarianFriendly]",621.0,4.0,2.0,84.0,/Restaurant_Review-g187309-d6864963-Reviews-Au...,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39975,Milan,"[Italian, VegetarianFriendly, VeganOptions, Gl...",500.0,4.5,2.0,79.0,/Restaurant_Review-g187849-d2104414-Reviews-Ro...,1,0,0,...,0,0,0,0,0,0,0,0,0,0
39976,Paris,"[French, American, Bar, European, VegetarianFr...",6341.0,3.5,2.0,542.0,/Restaurant_Review-g187147-d1800036-Reviews-La...,1,1,0,...,0,0,0,1,0,0,0,0,0,0
39977,Stockholm,"[Japanese, Sushi]",1652.0,4.5,2.0,4.0,/Restaurant_Review-g189852-d947615-Reviews-Sus...,0,0,0,...,0,0,0,0,0,0,1,0,0,0
39978,Warsaw,"[Polish, European, EasternEuropean, CentralEur...",641.0,4.0,2.0,70.0,/Restaurant_Review-g274856-d1100838-Reviews-Ho...,1,1,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
#X = df.drop(['id', 'rating'], axis = 1)
from sklearn.model_selection import train_test_split
X = df.drop(['city', 'rating', 'price_range', 'cuisine', 'url'], axis=1)
y = df['rating']

# Загружаем специальный инструмент для разбивки:

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) 

In [14]:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics  # инструменты для оценки точности модели

# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [15]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются  
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.21703751875937968
